Welcome to the Bancor Governance Forum. In this forum, the BancorDAO (Decentralized Autonomous Organization) proposes and discusses token Whitelistings, Trading liquidity limits, Fee changes, Bancor Improvement Proposals, and others.

Please head on to the [Information and Templates](https://gov.bancor.network/c/information-and-templates/16) category for more information on the DAO process.

Voting happens on the [BancorDAO Snapshot Page](https://vote.bancor.network/).

[Proposal: Limit on-curve liquidity to max(520 x 7 day fees, 100k BNT)](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876)
=============================================================================================================================================================================

[LEVEL 1 Under Review](https://gov.bancor.network/c/level-1-under-review/14)


[![](https://dub2.discourse-cdn.com/standard20/user_avatar/gov.bancor.network/thedavidmeister/90/129_2.png)](https://gov.bancor.network/u/thedavidmeister)

[thedavidmeister](https://gov.bancor.network/u/thedavidmeister)

TLDR:

-   We pay IL on 100% of TKN available for trading
-   We get diminishing returns on new fees per unit of trading liquidity
-   If IL > fees then the protocol dies (and is already in crisis due to existing deficits)
-   B3 has the unique ability to set a cap on trading liquidity
-   Taking capital off-curve has a dual benefit of limiting IL exposure AND making it available to other revenue opportunities (native staking etc.)
-   Currently the protocol deficit increases when ETH price increases by ~1.5x annually relative to BNT, making some assumptions, we could lift this to the protocol being profitable up to a ~4x ETH annual moon relative to BNT by limiting the on-curve trading liquidity to 520x 7 day trading fees + implementing some modest staking model (or similar)
-   We should set a simple cap based on the data we have and commission a more sophisticated model in the near future

Proposal:

When we look at the IL curve:

![image](https://aws1.discourse-cdn.com/standard20/uploads/bancordao/optimized/2X/e/e0d10c4f716b77cc23de68cb160db8c6e9dc28e1_2_690x387.png)


We see that the losses due to IL are a % of all the capital deployed on the trading curve.

If TKN moons 2x relative to BNT then the IL is ~10%, a 4x moon has ~20% IL, etc.

Meanwhile there are only so many fees due to trade volume across all of defi and almost all trades through bancor currently are based on arbitrage and aggregators only. That is to say, there is negligible trading being done directly on bancor due to retail etc. as that market is largely monopolized by aggregators and a few platforms that take the lion's share of human traders such as uniswap.

In considering this proposal, please consider mainly how arb bots and aggregators will respond, not human traders, as the former is 10x+ the latter in terms of protocol fee generation.

Import Data

In [0]:
import pandas as pd

filepath = 'https://bancorml.s3.us-east-2.amazonaws.com/preprocessed_tradingliquidity_vs_deficit.csv'
df = pd.read_csv(filepath)

Calculate Rolling Average Fee Revenue

In [0]:
# For 7 day rolling averages
df['trade_revenue_real_rolling_7day_avg'] =  df.groupby(['date', 'targetSymbol'])['trade_revenue_real'].rolling(7, min_periods=1).mean().reset_index(drop=True)
df['trade_revenue_usd_rolling_7day_avg'] = df.groupby(['date', 'targetSymbol'])['trade_revenue_usd'].rolling(7, min_periods=1).mean().reset_index(drop=True)

# For 30 day rolling averages
df['trade_revenue_real_rolling_30day_avg'] = df.groupby(['date', 'targetSymbol'])['trade_revenue_real'].rolling(30, min_periods=1).mean().reset_index(drop=True)
df['trade_revenue_usd_rolling_30day_avg'] = df.groupby(['date', 'targetSymbol'])['trade_revenue_usd'].rolling(30, min_periods=1).mean().reset_index(drop=True)
df

Out[41]:

,date,targetSymbol,trade_count,targetAmount_real,targetAmount_real_usd,trade_revenue_real,trade_revenue_usd,bntprice,surplus_tkn,surplus_bnt,surplus_usd,surplus_perc,trade_revenue_real_rolling_7day_avg,trade_revenue_usd_rolling_7day_avg,trade_revenue_real_rolling_30day_avg,trade_revenue_usd_rolling_30day_avg
0,2022-04-20,dai,3,9.205342,9.140794,0.018448,0.018318,2.296663,-9.221023e+00,-4.026950e+00,-9.248547e+00,-0.000054,0.018448,0.018318,0.018448,0.018318
1,2022-04-21,bnt,3,2.987996,6.991911,0.005988,0.014012,2.284838,-2.999949e+00,-2.999949e+00,-6.854397e+00,-0.000014,0.005988,0.014012,0.005988,0.014012
2,2022-04-22,bnt,1,1.342914,3.021557,0.002691,0.006055,2.231059,-4.806735e+00,-4.806735e+00,-1.072411e+01,-0.000011,0.002691,0.006055,0.002691,0.006055
3,2022-04-22,dai,2,618.054738,601.526021,1.238587,1.205463,2.231059,-6.283286e+02,-2.744009e+02,-6.122045e+02,-0.001828,1.238587,1.205463,1.238587,1.205463
4,2022-04-23,bnt,1,1136.475338,2511.610497,2.277506,5.033288,2.227919,-6.742615e+00,-6.742615e+00,-1.502200e+01,-0.000016,2.277506,5.033288,2.277506,5.033288
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1760,2022-07-30,usdt,3,8048.896705,8006.239119,81.301985,80.871100,0.564534,-1.262901e+06,-2.190509e+06,-1.236617e+06,-0.407160,81.301985,80.871100,81.301985,80.871100
1761,2022-07-30,vbnt,12,49419.825171,23492.709503,752.586170,357.756998,0.564534,-1.939145e+05,-1.594473e+05,-9.001343e+04,-0.056916,752.586170,357.756998,752.586170,357.756998
1762,2022-07-30,wbtc,24,3.114918,74919.134757,0.006242,150.135406,0.564534,-1.726202e+02,-7.197113e+06,-4.063016e+06,-0.478564,0.006242,150.135406,0.006242,150.135406
1763,2022-07-30,woo,47,633485.015403,143133.895043,1269.509049,286.841473,0.564534,-2.137629e+06,-8.456805e+05,-4.774155e+05,-0.267604,1269.509049,286.841473,1269.509049,286.841473


[](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876/1#current-situation-1)Current situation
-----------------------------------------------------------------------------------------------------------------------------------------------

Let's look at how that translates to the current state of play for ETH.

There is 8807 ETH on the trading curve right now:

![image](https://aws1.discourse-cdn.com/standard20/uploads/bancordao/original/2X/7/7fc569f4bf2f391578b84463ea82683fd815b20d.png)

Based on the above, set `oncurve_eth` and `token_symbol` constants:

In [0]:
oncurve_eth = 8800
token_symbol = 'eth'

Get the latest date in the dataset:

In [0]:
latest_date = df.sort_values('date', ascending=False)['date'].values[0]
latest_date

Out[76]: '2022-07-30'

30 day rolling average ETH fees are:

In [0]:
eth_rolling_30day_real_avg = df[(df['date']==latest_date) & (df['targetSymbol']==token_symbol)]['trade_revenue_real_rolling_30day_avg'].values[0]

eth_rolling_30day_usd_avg = df[(df['date']==latest_date) & (df['targetSymbol']==token_symbol)]['trade_revenue_usd_rolling_30day_avg'].values[0]

print(f'\033[1m 30 day rolling avg: {eth_rolling_30day_real_avg} (real/day) \033[1m')
print(f'\033[1m 30 day rolling avg: {eth_rolling_30day_usd_avg} (usd/day) \033[1m')

30 day rolling avg: 0.3133156103817142 (real/day) 
 30 day rolling avg: 533.9804979857236 (usd/day)

Estimated 7 day ETH fees are:

In [0]:
estimated_7day_eth_fees_real = eth_rolling_30day_real_avg * 7
estimated_7day_eth_fees_usd = eth_rolling_30day_usd_avg * 7

print(f'\033[1m Estimated 7 day ETH fees: {estimated_7day_eth_fees_real} (real) \033[1m')
print(f'\033[1m Estimated 7 day ETH fees: {estimated_7day_eth_fees_usd} (usd) \033[1m')

Estimated 7 day ETH fees: 2.193209272671999 (real) 
 Estimated 7 day ETH fees: 3737.863485900065 (usd)

Comparing the above to prior naive estimate (limited data):

![image](https://aws1.discourse-cdn.com/standard20/uploads/bancordao/original/2X/3/3b2d3eb390e5564193283285bad0d7a4a333cf53.png)

If we annualize the estimated fees we get:

In [0]:
eth_annualized = 52 * estimated_7day_eth_fees_real
eth_annualized_percent_of_oncurve = (eth_annualized / oncurve_eth) * 100

print(f'\033[1m ETH Fees annualized: {eth_annualized} (real) \033[1m')
print(f'\033[1m ETH Fees annualized as a percent of ETH on curve: {eth_annualized_percent_of_oncurve} (%) \033[1m')

ETH Fees annualized: 114.04688217894396 (real) 
 ETH Fees annualized as a percent of ETH on curve: 1.2959872974879996 (%)

Setting aside mechanics like leveraged vortexing for the moment, the baseline is that the protocol will break even on ETH if IL on 8800 ETH is 1.29% or less, relative to BNT.

That means that if ETH increases by just 50% relative to BNT over a year, all else equal, the protocol loses ETH and the deficit increases.

[](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876/1#proposed-2)Proposed
-----------------------------------------------------------------------------------------------------------------------------

Let's limit the ETH pool size to 520x the 7 day fees, i.e. an APR of 10% all else equal.

This gives us a limited trading liquidity of:

In [0]:
limited_pool_size_520 = int(round(520 * estimated_7day_eth_fees_real, 0))
print(f'\033[1m {limited_pool_size_520} (ETH) \033[1m')

1140 (ETH)

If we assume that the fees generated on the limited trading liquidity are not significantly less than the fees on the current on-curve liquidity then this gives us an APR of 10% on ETH.

We spent all of bancor v2.1 with caps on trading pools showing that 10% APR is sustainable for many TKN, but the reality of how many fees we would miss on a 2400 ETH pool vs. an 8800 ETH pool is something that we would have to measure and monitor.

Continuing with the assumption of comparable fees with the limited pool...

This now means that a 2x in ETH vs. BNT costs the protocol 240 ETH instead of 880 ETH. That is to say, over 1 year we can expect the protocol to break even on the 10% APR even if ETH moons 2x relative to BNT.

We also have 6400 ETH freed up to generate fees. Native staking on ETH yields about 3%.

6400 ETH x 0.03 = 192 ETH ~= 200 ETH

Assuming we do the most conservative thing other than doing nothing at all, use the native ETH staking mechanism, we now have a budget of 200 ETH from staking 6400 ETH and 240 ETH from a 10% APR on 2400 ETH to cover the IL on 2400 ETH.

With 440 ETH we can cover up to ~18% IL on 2400 ETH which allows ETH to annually moon ~4x relative to BNT

Clearly a 4x annual moon is a significantly larger safety margin than a 1.5x annual moon.

All we need to do is:

-   Limit pool to 520x the 7 day fees to give a minimum 10% APR
-   Move the off-curve liquidity into a conservative yield generating mechanism offering 3% e.g. native staking

Generally we could and should apply this 520x cap to all TKN on the platform as a rule of thumb and then backfill more sophisticated analytics over time.

For many TKN we might run into a situation where the 7 day fees are too small to support trading of any size, so we can set a minimum, which right now can be somewhat arbitrary like 100k BNT.

[](https://gov.bancor.network/t/proposal-limit-on-curve-liquidity-to-max-520-x-7-day-fees-100k-bnt/3876/1#voting-3)Voting
-------------------------------------------------------------------------------------------------------------------------

YES: Cap all on-curve TKN trading liquidity to 520x their rolling 7 day fees

-   Frequency and level of automation of the cap management TBD and improved over time
-   Commit to alternative revenue generation for the off-curve capital
-   Commit to monitoring and ongoing analysis that is more sophisticated than what is presented above

NO: Continue to attempt to cover double digit annual % IL with single digit % annual TKN fees